In [1]:
# Interpola el delta de temperatura
# y precipitación para que sea compatible
# con la resolución de CHIRTS y CHIRPS.

import os

import rioxarray

import pandas as pd
import numpy as np

import xarray as xr

In [17]:
vd = 0
path = "../../../datos/CHIRTS/"
Delta = ["../../../datos/AIRCC-CLIM/",
    "../../../resultados/AIRCC-CLIM/"]
months = ["January", "February", "March", "April",
    "May", "June", "July", "August",
    "September", "October", "November", "December"]
years = [2030, 2050, 2070]
RCP = ["RCP4p5", "RCP8p5"]
vars = ["Tmax", "Tmin", "Tmean", "Pre"]
var_delta = ["temp", "prec"]
var_base = ["Temp", "Prec_perc"]
lon_min = -118.390
lon_max =  -86.660
lat_min =   14.566
lat_max =   32.737

# Si no existe la carpeta, la crea.
if not os.path.exists(Delta[1]):
    os.mkdir(Delta[1])
for i in RCP:
    if not os.path.exists(Delta[1] + i):
        os.mkdir(Delta[1] + i)
    if not os.path.exists(Delta[1] + i + "_base"):
        os.mkdir(Delta[1] + i + "_base")

# Archivo base para interpolar.
ds = xr.open_dataset(path +
    "Tmax.1983.01.01.tif.nc" )

In [3]:
fpath_d_base = []
fpath_r_base = []
y = 2005

# Crea una lista con todas las direcciones
# de los datos y de los resultados.
for i in range(len(var_delta)):
    for j in RCP:
        for m_n, m in enumerate(months):
            m_n += 1

            fpath_d_base.append(Delta[0] + j + "_base/" +
            var_base[i] + "_SchneiderThompson_Stochastic_"
            + j + "_" + m + "_" + str(y) + ".nc")

            fpath_r_base.append(Delta[1] + j + "_base/" +
            var_delta[i] + "_tol_Stochastic_"
            + j + "_" + f"{m_n:02d}" + "_"
            + str(y) + ".nc")

# Interpola, usa np.nan como datos
# faltante y guarda el archivo.
for i in range(len(fpath_d_base)):
    d = xr.open_dataset( fpath_d_base[i] )
    d = d.interp_like(ds)
    # Recortamos la caja que delimita a México.
    d = d.rio.set_spatial_dims("lon", "lat")
    d = d.rio.write_crs("EPSG:4326")
    d = d.rio.clip_box( lon_min,
        lat_min, lon_max, lat_max )
    d = d.drop_vars("spatial_ref")
    if fpath_d_base[i][38:42] == var_base[0]: n = 0
    else: n = 1
    d = d.rename({var_base[n]: var_delta[n]})
    d.where(d > -9000).to_netcdf( fpath_r_base[i] )

In [4]:
fpath_d = []
fpath_r = []

# Crea una lista con todas las direcciones
# de los datos y de los resultados.
for v in var_delta:
    for j in RCP:
        for y in years:
            for m_n, m in enumerate(months):
                m_n += 1

                fpath_d.append(Delta[0] + j + "/" +
                v + "_tol_Stochastic_" +
                j + "_" + m + "_" + str(y) + ".nc")

                fpath_r.append(Delta[1] + j + "/"
                + v + "_tol_Stochastic_"
                + j + "_" + f"{m_n:02d}" + "_"
                + str(y) + ".nc")

# Interpola, usa np.nan como dato
# faltante y guarda el archivo.
for i in range(len(fpath_d)):
    d = xr.open_dataset( fpath_d[i] )
    d = d.interp_like(ds)
    # Recortamos la caja que delimita a México.
    d = d.rio.set_spatial_dims("lon", "lat")
    d = d.rio.write_crs("EPSG:4326")
    d = d.rio.clip_box( lon_min,
        lat_min, lon_max, lat_max )
    d = d.drop_vars("spatial_ref")
    d = d.where(d > -9000)
    # Quizás sería más eficiente abrir todos los
    # archivos base, concatenarlos y acceder a
    # cada mes del Dataset, pero entonces se
    # dificulta trabajar con una lista de la
    # dirección del archivo y habría que tener
    # ciclos anidados con RCP, var_delta, y el mes,
    # lo que a su vez restaría eficiencia.
    b = xr.open_dataset( fpath_r[i][:37]
        + "_base" + fpath_r[i][37:-7]
        + "2005.nc" )
    # Quitamos el efecto climático ya
    # presente en la climatología 1995-2014.
    if fpath_r[i][38:42] == var_delta[0]:
        d -= b
    else: 
        d = ( 1 + d / 100 ) / ( 1 + b / 100 ) - 1
    d.to_netcdf( fpath_r[i] )

In [19]:
SSP = "../../../resultados/SSP/"
esc = ["SSP5"]
var_soc = ["GDP", "POP"]
fname = [var_soc[0]  + "_" + esc[0] + ".nc",
    var_soc[1]  + "_" + esc[0] + ".nc"]

i = 0

d = xr.open_dataset(SSP + fname[i])
d = d.assign_coords( {
    "time": pd.date_range("2008-01-01", 
    "2099-01-01", freq = "1Y"),
    "lon": np.arange(0, 360, 0.5),
    "lat": np.arange(-90, 90, 0.5),
    } )
#d = d.interp_like(ds)
# Recortamos la caja que delimita a México.
#d = d.rio.set_spatial_dims("lon", "lat")
#d = d.rio.write_crs("EPSG:4326")
#d = d.rio.clip_box( lon_min,
#    lat_min, lon_max, lat_max )
#d = d.drop_vars("spatial_ref")
#d = d.where(d > -9000)

d